### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).





In [1]:
import numpy as np
def psat(i):
    t=20
    a1=8.07131,7.43155
    a2=1730.63,1554.679
    a3=233.426,240.337
    power=(np.log(10)*(a1[i]-(a2[i]/(t+a3[i]))))
    return np.exp(power)
psatw=psat(0)
psat1=psat(1)
print(psatw,psat1)

17.47325208459706 28.82409952740525


In [18]:
def eqp(a):
    x1 =0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
    p =28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5
    sum=0.0
    for i in range (11):
        t1=x1[i]*(t.exp(psatw*a[0]*((a[1]*(1-x1[i])/(a[0]*x1[i]+a[1]*(1-x1[i])))**2)))
        t2=(1-x1[i])*(t.exp(psat1*a[1]*((a[0]*x1[i]/(a[0]*x1[i]+a[1]*(1-x1[i])))**2)))
        sum=sum+(t1+t2-p[i])**2
    return sum
# eqp([1,1])

In [40]:
# Without line search

import torch as t
from torch.autograd import Variable

x = Variable(t.tensor([1.0, 1.0]), requires_grad=True)

# Fix the step size
a = 0.01

# Start gradient descent
for i in range(1000):  # TODO: change the termination criterion
    loss = eqp(x)
    loss.backward()
    
    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        x -= a * x.grad
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        x.grad.zero_()
        
print(x.data.numpy())
print(loss.data.numpy())

tensor([0., 0.])
[-1.7776660e+17 -7.1091075e+17]
12034.761


In [39]:
# With line search

import torch as t
from torch.autograd import Variable
x = Variable(t.tensor([1.0, 1.0]), requires_grad=True)        
loss=eqp(x)
loss.backward()
soln=[x]
error=t.linalg.norm(x.grad)
a=0.01
eps=1e-3

def line_search(x,d):
  a=1
  def phi(a,x,d):
    return eqp(x)+a*0.8*np.dot(x.grad,d)

  while phi(a,x,d)<eqp(x+a*d):
    a=0.5*a
  
  return a

while error>=eps:
  d=-x.grad
  a=line_search(x,d)
  x=x+a*d
  soln.append(x)
  error=t.linalg.norm(x.grad)

print(soln)
# print(x.data.numpy())
print(loss.data.numpy())
print(x.grad)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in multiply
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.


TypeError: ignored